# 🚀 Kaggle Buffer Test Runner - All-in-One

This notebook loads vLLM, registers it with the backend, and runs buffer tests.
**Run all cells in order - no separate notebook needed!**

## What this does:
1. ✅ Installs vLLM dependencies
2. ✅ Loads Qwen 14B AWQ model on Kaggle GPUs
3. ✅ Clones repo and configures git
4. ✅ Registers vLLM with backend (for responses, summarization, AND judging)
5. ✅ Runs buffer tests (sizes: 5, 10, 20, 40)
6. ✅ Auto-pushes results to GitHub after each buffer

## Important:
- This is a **single notebook** - no need for separate notebooks
- vLLM model stays in memory for all operations
- No server needed - tests use direct Python imports

In [1]:
# Cell 1: Install vLLM dependencies
print("="*60)
print("📦 INSTALLING vLLM DEPENDENCIES")
print("="*60)

! uv pip uninstall -q --system 'tensorflow'
! uv pip install -q --system 'vllm' 'triton==3.2.0' 'logits-processor-zoo' 'numpy<2'

print("✅ Dependencies installed")
print("="*60)

📦 INSTALLING vLLM DEPENDENCIES
✅ Dependencies installed
✅ Dependencies installed


In [2]:
# Cell 2: Import libraries
import os
import shutil
import subprocess
import sys
import numpy as np
import pandas as pd
import torch
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor

print("="*60)
print("📚 LIBRARIES IMPORTED")
print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"🎮 GPUs available: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
print("="*60)

INFO 12-21 11:40:41 [__init__.py:239] Automatically detected platform cuda.
📚 LIBRARIES IMPORTED
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: True
🎮 GPUs available: 2
   GPU 0: Tesla T4
   GPU 1: Tesla T4
📚 LIBRARIES IMPORTED
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: True
🎮 GPUs available: 2
   GPU 0: Tesla T4
   GPU 1: Tesla T4


In [3]:
# Cell 3: Load Kaggle secrets and set environment
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

# Load all secrets
os.environ["GITHUB_TOKEN"] = user_secrets.get_secret("GITHUB_TOKEN")
os.environ["GROQ_API_KEY"] = user_secrets.get_secret("GROQ_API_KEY")
os.environ["HuggingFACEHUB_access_token"] = user_secrets.get_secret("HuggingFACEHUB_access_token")
os.environ["LANGCHAIN_API_KEY"] = user_secrets.get_secret("LANGCHAIN_API_KEY")

# Set vLLM configuration
os.environ["LLM_BACKEND"] = "vllm"
model_path = "/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1"
os.environ["VLLM_MODEL_PATH"] = model_path
os.environ["VLLM_USE_V1"] = "0"

print("="*60)
print("🔐 SECRETS AND CONFIGURATION LOADED")
print("="*60)
print(f"✅ GITHUB_TOKEN: {os.environ['GITHUB_TOKEN'][:4]}...{os.environ['GITHUB_TOKEN'][-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print(f"✅ VLLM_MODEL_PATH: {model_path}")
print("="*60)

🔐 SECRETS AND CONFIGURATION LOADED
✅ GITHUB_TOKEN: gith...tWfg
✅ LLM_BACKEND: vllm
✅ VLLM_MODEL_PATH: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1


In [4]:
# Cell 4: Load vLLM model on Kaggle GPUs (takes 2-3 minutes)
print("="*60)
print("🚀 LOADING vLLM MODEL")
print("="*60)
print(f"📂 Model: {model_path}")
print(f"🎮 GPUs: {torch.cuda.device_count()}")
print("⏳ This takes 2-3 minutes...")
print("="*60)

llm = vllm.LLM(
    model_path,
    quantization='awq',
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.91,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    enable_prefix_caching=True
)
tokenizer = llm.get_tokenizer()

print("\n" + "="*60)
print("✅ vLLM MODEL LOADED SUCCESSFULLY!")
print("="*60)
print(f"   Memory per GPU: ~{torch.cuda.get_device_properties(0).total_memory / 1024**3 * 0.91:.1f}GB used")
print("="*60)

🚀 LOADING vLLM MODEL
📂 Model: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
🎮 GPUs: 2
⏳ This takes 2-3 minutes...
INFO 12-21 11:41:11 [config.py:717] This model supports multiple tasks: {'reward', 'embed', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
INFO 12-21 11:41:11 [config.py:717] This model supports multiple tasks: {'reward', 'embed', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 12-21 11:41:12 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-21 11:41:12 [config.py:1770] Defaulting to use mp for distributed inference
WARNING 12-21 11:41:12 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 12-21 11:41:12 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-21 11:41:12 [config.py:1770] Defaulting to

[W1221 11:41:30.571830905 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1221 11:41:30.572657524 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-21 11:41:30 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=167) INFO 12-21 11:41:30 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=167) INFO 12-21 11:41:30 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-21 11:41:30 [pynccl.py:69] vLLM is using nccl==2.21.5
(VllmWorkerProcess pid=167) INFO 12-21 11:41:30 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-21 11:41:30 [pynccl.py:69] vLLM is using nccl==2.21.5


[W1221 11:41:30.842687332 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1221 11:41:30.843351650 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-21 11:41:31 [custom_all_reduce_utils.py:206] generating GPU P2P access cache in /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-21 11:41:56 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=167) INFO 12-21 11:41:56 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-21 11:41:56 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=167) INFO 12-21 11:41:56 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-21 11:41:56 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[1], buffer_handle=(1, 4194304, 6, 'psm_db2efac3'), local_subscribe_addr='ipc:///tmp/02016430-01db-4f36-b7cf-4ffd5f87d7f6', remote_subscri

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(VllmWorkerProcess pid=167) INFO 12-21 11:42:43 [loader.py:458] Loading weights took 47.27 seconds
INFO 12-21 11:42:43 [loader.py:458] Loading weights took 47.38 seconds
INFO 12-21 11:42:44 [model_runner.py:1140] Model loading took 4.6720 GiB and 47.670479 seconds
(VllmWorkerProcess pid=167) INFO 12-21 11:42:43 [model_runner.py:1140] Model loading took 4.6720 GiB and 47.558244 seconds
INFO 12-21 11:42:44 [model_runner.py:1140] Model loading took 4.6720 GiB and 47.670479 seconds
(VllmWorkerProcess pid=167) INFO 12-21 11:42:43 [model_runner.py:1140] Model loading took 4.6720 GiB and 47.558244 seconds
(VllmWorkerProcess pid=167) INFO 12-21 11:42:54 [worker.py:287] Memory profiling takes 10.04 seconds
(VllmWorkerProcess pid=167) INFO 12-21 11:42:54 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.91) = 13.41GiB
(VllmWorkerProcess pid=167) INFO 12-21 11:42:54 [worker.py:287] model weights take 4.67GiB; non_torch_memory takes 0.12GiB; 

In [5]:
# Cell 5: Clone repository and configure git
REPO_URL = "https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
REPO_DIR = "Subchat-Trees"
BRANCH = "kaggle-run"

print("="*60)
print("📥 CLONING REPOSITORY")
print("="*60)

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing {REPO_DIR} directory...")
    shutil.rmtree(REPO_DIR)

# Clone with LFS skip to save bandwidth
clone_env = os.environ.copy()
clone_env["GIT_LFS_SKIP_SMUDGE"] = "1"

result = subprocess.run(
    ["git", "clone", "-b", BRANCH, "--single-branch", REPO_URL, REPO_DIR],
    capture_output=True,
    text=True,
    env=clone_env
)

if result.returncode == 0:
    print(f"✅ Cloned {BRANCH} branch!")
    
    # Pull LFS files for scenarios
    os.chdir(REPO_DIR)
    subprocess.run(
        ["git", "lfs", "pull", "--include=backend/dataset/scenarios/*.json"],
        capture_output=True,
        text=True
    )
    print("✅ Pulled scenario files from Git LFS")
    
    # Configure git identity
    subprocess.run(["git", "config", "user.name", "moonmehedi"], check=True)
    subprocess.run(["git", "config", "user.email", "the.mehedi.hasan.moon@gmail.com"], check=True)
    print("✅ Git identity configured")
    
    os.chdir("..")
else:
    print(f"❌ Clone failed: {result.stderr}")

print("="*60)

📥 CLONING REPOSITORY
✅ Cloned kaggle-run branch!
✅ Pulled scenario files from Git LFS
✅ Git identity configured
✅ Cloned kaggle-run branch!
✅ Pulled scenario files from Git LFS
✅ Git identity configured


In [6]:
# Cell 6: Register vLLM with backend
sys.path.insert(0, os.path.join(REPO_DIR, "backend"))

from src.services.vllm_client import VLLMClient

print("="*60)
print("🔗 REGISTERING vLLM WITH BACKEND")
print("="*60)

VLLMClient.set_model(llm)

print(f"✅ vLLM registered: {VLLMClient.is_available()}")
print("   ✅ Response generation will use vLLM")
print("   ✅ Summarization will use vLLM")
print("   ✅ Judge/Classification will use vLLM")
print("="*60)

🔗 REGISTERING vLLM WITH BACKEND
✅ vLLM model registered: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
✅ vLLM registered: True
   ✅ Response generation will use vLLM
   ✅ Summarization will use vLLM
   ✅ Judge/Classification will use vLLM


In [7]:
# Cell 7: Install backend requirements
print("="*60)
print("📦 INSTALLING BACKEND REQUIREMENTS")
print("="*60)

! pip install -q -r /kaggle/working/Subchat-Trees/backend/requirements.txt

print("✅ Backend requirements installed")
print("="*60)

📦 INSTALLING BACKEND REQUIREMENTS


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 47.8 MB/s eta 0:00:0000:0100:01   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/4.2 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [8]:
# Cell 8: Quick test to verify vLLM integration works
from src.services.simple_llm import SimpleLLMClient
from src.models.tree import TreeNode

print("="*60)
print("🧪 QUICK INTEGRATION TEST")
print("="*60)

llm_client = SimpleLLMClient()
root = TreeNode(node_id="test", title="Test", buffer_size=5, llm_client=llm_client)
root.buffer.add_message("user", "Hello")

response = llm_client.generate_response(root, "What is 2+2?")
print(f"✅ Test response: {response[:100]}...")
print("✅ vLLM integration working!")
print("="*60)

✅ Using vLLM backend with Kaggle GPU: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
🧪 QUICK INTEGRATION TEST
🔧 LLM_BACKEND configured as: 'vllm'
✅ vLLM connected for RESPONSES: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
✅ vLLM will be used for SUMMARIZATION: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
📊 Buffer size: 5 messages | Summarization will trigger every 5 messages
📋 Buffer (1/5): Last 3 messages (full log in file)
   1. [user] Hello
*******************context*********************
 [{'role': 'system', 'content': 'You are participating in a multi-topic, multi-turn evaluation where topics persist independently of conversational order. Topics are introduced using the format topic_name : user query, and sub-topics using topic_name_subtopic_name : user query; these topic labels remain available for future reference. For every user query, you must analyze its semantic meaning and select the previously introduced topic or sub-topic it most strongly refers t

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

✅ Test response: general_greeting: Hello! How can I assist you today?...
✅ vLLM integration working!


In [ ]:
# Cell 9: RUN BUFFER TESTS (SERVERLESS - no HTTP server needed!)
# Using exec() to run in the SAME process so it can access the loaded vLLM model

print("="*60)
print("🚀 RUNNING SERVERLESS BUFFER TESTS")
print("="*60)
print("📊 Testing buffer sizes: 5, 10, 20, 40")
print("📄 Datasets: 6c4992f0aed04dd3bf9a4bc225bb4fb0_structured.json")
print("            8d10c143f8fc4a7599a5a18778fec112_structured.json")
print("📤 Results will auto-push to GitHub after each buffer")
print("⏳ This may take several hours depending on scenario count")
print("✅ NO SERVER NEEDED - using direct Python imports!")
print("✅ Max output tokens: 300")
print("="*60)

os.chdir("/kaggle/working/Subchat-Trees/backend")

# Import and run the serverless test runner with specific datasets
import sys
sys.path.insert(0, "/kaggle/working/Subchat-Trees/backend")
sys.path.insert(0, "/kaggle/working/Subchat-Trees/backend/dataset")

from dataset.kaggle_serverless_runner import ServerlessTestRunner

# Create runner
runner = ServerlessTestRunner()
runner.test_mode = "both"  # Run both baseline and system tests

# Datasets to test
scenario_files = [
    "6c4992f0aed04dd3bf9a4bc225bb4fb0_structured.json",
    "8d10c143f8fc4a7599a5a18778fec112_structured.json"
]

# Buffer sizes to test
buffer_sizes = [5, 10, 20, 40]

print(f"📄 Running scenarios: {scenario_files}")
print(f"📦 Buffer sizes: {buffer_sizes}")

# Run the comparison
runner.run_buffer_comparison(
    scenario_files,
    buffer_sizes=buffer_sizes
)

🚀 RUNNING SERVERLESS BUFFER TESTS
📊 Testing buffer sizes: 5, 10, 20, 40
📄 Datasets: 6c4992f0aed04dd3bf9a4bc225bb4fb0_structured.json
            8d10c143f8fc4a7599a5a18778fec112_structured.json
📤 Results will auto-push to GitHub after each buffer
⏳ This may take several hours depending on scenario count
✅ NO SERVER NEEDED - using direct Python imports!
✅ Max output tokens: 300
🔧 LLM_BACKEND configured as: 'vllm'
✅ vLLM connected for RESPONSES: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
✅ vLLM will be used for SUMMARIZATION: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
🔧 Kaggle detected: Using writable path /kaggle/working/chroma_db
🔧 Kaggle detected: Using writable path /kaggle/working/chroma_db


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

🔍 LLM Judge [Step 19]: yes -> TP
[11:48:50] [INFO]   ✅ Classification: TP (llm)
[11:48:51] [INFO] 
[Step 20] Context: ai_consciousness_friendship
[11:48:51] [INFO]   💬 User: Do you foresee a day that computer programs may experience human emotions?
📦 Archived message: Do you foresee a day that computer programs may experience h... (ID: a5a4e036-5075-4a91-a934-91957f4e99cb_1766317731.1548696)
📚 Vector store: 39 messages across 1 conversations (logged to file)
💾 Indexed: [user] Do you foresee a day that computer progr...
🔄 Buffer full - evicting: ai_consciousness_friendship: Currently, ...
📋 Buffer (5/5): Last 3 messages (full log in file)
   1. [user] Would you like to have the ability to experience e...
   2. [assistant] ai_consciousness_friendship: As an artificial inte...
   3. [user] Do you foresee a day that computer programs may ex...
*******************context*********************
 [{'role': 'system', 'content': 'You are participating in a multi-topic, multi-turn evaluation where

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

📦 Archived message: ai_consciousness_friendship: The possibility of computer pro... (ID: a5a4e036-5075-4a91-a934-91957f4e99cb_1766317736.5361035)
📚 Vector store: 40 messages across 1 conversations (logged to file)
💾 Indexed: [assistant] ai_consciousness_friendship: The possibi...
🔄 Buffer full - evicting: Your answer became super slow on my scre...


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
# Cell 10: Completion and cleanup (optional)
print("="*60)
print("✅ BUFFER TESTS COMPLETE!")
print("="*60)
print("📤 All results have been pushed to GitHub")
print("📊 Check the kaggle_logs/ directory in your repo")
print("")
print("💡 You can now stop the kernel to save GPU quota")
print("   Uncomment the line below to auto-shutdown:")
print("="*60)

# Uncomment to force shutdown and save GPU quota:
# import os; os._exit(0)